In [1]:
import os
import cv2
import pickle
from tqdm import tqdm
import numpy as np
import skimage.io
from sklearn.cluster import KMeans
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern

In [2]:
src1 = 'Data/train/flooded/'
src2 = 'Data/train/non-flooded/'
image_paths_1 = os.listdir(src1)
image_paths_1 = [src1+ s for s in image_paths_1]
image_paths_2 = os.listdir(src2)
image_paths_2 = [src2+ s for s in image_paths_2]
image_paths = image_paths_1 + image_paths_2
print(len(image_paths))

644


In [3]:
# Define parameters for LBP feature extraction
radius = 1
n_points = 8
METHOD = 'uniform'

In [4]:
print(len(image_paths))

644


In [6]:
grey_images = [ [] ] * len(image_paths)
lbp_features = [ [] ] * len(image_paths)
lbp_histograms = [ [] ] * len(image_paths)
i = 0
for image_path in tqdm(image_paths):
    image = skimage.io.imread(image_path)
    grey_image = rgb2gray(image)
    grey_images[i] = grey_image
    lbp = local_binary_pattern(grey_image, n_points, radius, METHOD)
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_features[i] = lbp
    lbp_histograms[i] = hist
    i += 1

  0%|          | 0/644 [00:00<?, ?it/s]/home/iten/anaconda3/envs/bd/lib/python3.11/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
 90%|█████████ | 581/644 [07:11<02:04,  1.97s/it]

: 

: 

In [5]:
len(lbp_features), lbp_features[0].shape

IndexError: list index out of range

In [ ]:
# Create a K-means model with 2 clusters
kmeans = KMeans(n_clusters=2)

# Fit the K-means model to the LBP features
kmeans.fit(lbp_features)

In [ ]:
# Predict the cluster labels for each image
cluster_labels = kmeans.predict(lbp_features)